# Introduction à la probabilité et aux statistiques
Dans ce carnet, nous allons explorer certains des concepts que nous avons précédemment abordés. De nombreux concepts de probabilité et de statistiques sont bien représentés dans les principales bibliothèques de traitement de données en Python, telles que `numpy` et `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Variables aléatoires et distributions
Commençons par tirer un échantillon de 30 valeurs à partir d'une distribution uniforme de 0 à 9. Nous allons également calculer la moyenne et la variance.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

Pour estimer visuellement combien de valeurs différentes se trouvent dans l’échantillon, nous pouvons tracer l'**histogramme** :


In [ ]:
plt.hist(sample)
plt.show()

## Analyse des données réelles

La moyenne et la variance sont très importantes lors de l'analyse de données du monde réel. Chargons les données sur les joueurs de baseball depuis [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Nous utilisons un package appelé [**Pandas**](https://pandas.pydata.org/) ici pour l'analyse des données. Nous parlerons plus en détail de Pandas et du travail avec les données en Python plus tard dans ce cours.

Calculons les valeurs moyennes pour l'âge, la taille et le poids :


In [ ]:
df[['Age','Height','Weight']].mean()

Concentrons-nous maintenant sur la taille, et calculons l'écart type et la variance :


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

En plus de la moyenne, il est judicieux d'examiner la valeur médiane et les quartiles. Ils peuvent être visualisés à l'aide d'un **diagramme en boîte** :


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

Nous pouvons également réaliser des diagrammes en boîte de sous-ensembles de notre ensemble de données, par exemple, regroupés par rôle du joueur.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Note** : Ce diagramme suggère qu'en moyenne, les joueurs de première base sont plus grands que les joueurs de deuxième base. Plus tard, nous apprendrons comment tester cette hypothèse de manière plus formelle, et comment démontrer que nos données sont statistiquement significatives pour le montrer.  

L'âge, la taille et le poids sont tous des variables aléatoires continues. Quelle est selon vous leur distribution ? Une bonne façon de le découvrir est de tracer l'histogramme des valeurs : 


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Distribution Normale

Créons un échantillon artificiel de poids qui suit une distribution normale avec la même moyenne et variance que nos données réelles :


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Puisque la plupart des valeurs dans la vie réelle suivent une distribution normale, nous ne devrions pas utiliser un générateur de nombres aléatoires uniformes pour générer des données d'échantillon. Voici ce qui se passe si nous essayons de générer des poids avec une distribution uniforme (générée par `np.random.rand`):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Intervalles de confiance

Calculons maintenant les intervalles de confiance pour les poids et tailles des joueurs de baseball. Nous allons utiliser le code [de cette discussion Stack Overflow](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data) :


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Test d'hypothèse

Explorons différents rôles dans notre ensemble de données sur les joueurs de baseball :


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Testons l'hypothèse que les premiers buteurs sont plus grands que les seconds buteurs. La manière la plus simple de le faire est de tester les intervalles de confiance :


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Nous pouvons voir que les intervalles ne se chevauchent pas.

Une manière statistiquement plus correcte de prouver l’hypothèse est d’utiliser un **test t de Student** :


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

Les deux valeurs retournées par la fonction `ttest_ind` sont :
* La p-value peut être considérée comme la probabilité que deux distributions aient la même moyenne. Dans notre cas, elle est très faible, ce qui signifie qu'il existe une forte preuve soutenant que les premiers buts sont plus grands.
* La t-value est la valeur intermédiaire de la différence de moyenne normalisée qui est utilisée dans le test t, et elle est comparée à une valeur seuil pour un niveau de confiance donné.


## Simulation d'une distribution normale avec le théorème central limite

Le générateur pseudo-aléatoire en Python est conçu pour nous donner une distribution uniforme. Si nous voulons créer un générateur pour une distribution normale, nous pouvons utiliser le théorème central limite. Pour obtenir une valeur distribuée normalement, nous allons simplement calculer la moyenne d'un échantillon généré uniformément.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Corrélation et Evil Baseball Corp

La corrélation nous permet de trouver des relations entre des séquences de données. Dans notre exemple ludique, imaginons qu'il existe une corporation de baseball maléfique qui paie ses joueurs en fonction de leur taille - plus le joueur est grand, plus il/elle reçoit d'argent. Supposons qu'il y ait un salaire de base de 1000 $, et un bonus additionnel de 0 à 100 $, selon la taille. Nous prendrons les vrais joueurs de la MLB, et calculerons leurs salaires imaginaires :


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Calculons maintenant la covariance et la corrélation de ces séquences. `np.cov` nous donnera une **matrice de covariance**, qui est une extension de la covariance à plusieurs variables. L'élément $M_{ij}$ de la matrice de covariance $M$ est une covariance entre les variables d'entrée $X_i$ et $X_j$, et les valeurs diagonales $M_{ii}$ sont la variance de $X_{i}$. De même, `np.corrcoef` nous donnera la **matrice de corrélation**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Une corrélation égale à 1 signifie qu'il existe une **relation linéaire** forte entre deux variables. Nous pouvons voir visuellement la relation linéaire en traçant une valeur par rapport à l'autre :


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

Voyons ce qui se passe si la relation n'est pas linéaire. Supposons que notre entreprise ait décidé de cacher la dépendance linéaire évidente entre les tailles et les salaires, et ait introduit une certaine non-linéarité dans la formule, comme `sin` :


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Dans ce cas, la corrélation est légèrement plus faible, mais elle reste assez élevée. Maintenant, pour rendre la relation encore moins évidente, nous pourrions vouloir ajouter un peu de hasard supplémentaire en ajoutant une variable aléatoire au salaire. Voyons ce qui se passe :


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Pouvez-vous deviner pourquoi les points s’alignent en lignes verticales comme ceci ?

Nous avons observé la corrélation entre un concept artificiellement conçu comme le salaire et la variable observée *taille*. Voyons également si les deux variables observées, comme la taille et le poids, sont également corrélées :


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

Malheureusement, nous n'avons obtenu aucun résultat - seulement quelques valeurs étranges `nan`. Cela est dû au fait que certaines des valeurs de notre série sont indéfinies, représentées comme `nan`, ce qui entraîne que le résultat de l'opération soit également indéfini. En regardant la matrice, on peut voir que `Weight` est la colonne problématique, car la corrélation avec elle-même entre les valeurs de `Height` a été calculée.

> Cet exemple montre l'importance de la **préparation des données** et du **nettoyage**. Sans données appropriées, nous ne pouvons rien calculer.

Utilisons la méthode `fillna` pour remplir les valeurs manquantes, et calculons la corrélation :


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

Il existe en effet une corrélation, mais pas aussi forte que dans notre exemple artificiel. En effet, si l'on regarde le nuage de points d'une valeur par rapport à l'autre, la relation serait beaucoup moins évidente :


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Conclusion

Dans ce carnet, nous avons appris comment effectuer des opérations de base sur les données pour calculer des fonctions statistiques. Nous savons maintenant comment utiliser un appareil mathématique et statistique solide afin de vérifier certaines hypothèses, et comment calculer des intervalles de confiance pour des variables arbitraires à partir d'un échantillon de données.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Clause de non-responsabilité** :  
Ce document a été traduit à l’aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d’assurer l’exactitude de la traduction, veuillez noter que les traductions automatiques peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue native doit être considéré comme la source faisant foi. Pour les informations critiques, il est recommandé de recourir à une traduction professionnelle effectuée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d’interprétations erronées résultant de l’utilisation de cette traduction.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
